In [84]:
%matplotlib inline

In [1]:
import time
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix as confu
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import grid_search
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier

In [2]:
training = pd.read_csv('kdd_10_per.csv')
testing= pd.read_csv('corrected', header=None)
attack_list = open('training_attack_types.txt')
attack_dic = {}

for line in attack_list.readlines():    
    detail = line.split(' ')
    if len(detail) == 1:
        break
    attack_type = detail[1][:-1]
    attack_name = detail[0]
    
    if attack_dic.has_key(attack_type):
        attack_dic[attack_type].append(attack_name)
    else:
        attack_dic[attack_type] = []
        attack_dic[attack_type].append(attack_name)

string_label_columns = ['flag','service','protocol_type'] 
testing.columns = training.columns
#for i in range(len(string_label_columns)):
#    del(training[string_label_columns[i]])

In [3]:
test_attack_dic = {}
test_attack_dic["apache2."] = 2 
test_attack_dic["back."] = 2 
test_attack_dic["buffer_overflow."] = 3 
test_attack_dic["ftp_write."] = 4 
test_attack_dic["guess_passwd."] = 4 
#test_attack_dic["httptunnel."] = 4 
test_attack_dic["httptunnel."] = 3 
test_attack_dic["imap."] = 4 
test_attack_dic["ipsweep."] = 1 
test_attack_dic["land."] = 2 
test_attack_dic["loadmodule."] = 3 
test_attack_dic["mailbomb."] = 2 
test_attack_dic["mscan."] = 1 
test_attack_dic["multihop."] = 4 
#test_attack_dic["multihop."] = 3 # note that this is a duplicate 
test_attack_dic["named."] = 4 
test_attack_dic["neptune."] = 2 
test_attack_dic["nmap."] = 1 
test_attack_dic["perl."] = 3 
test_attack_dic["phf."] = 4 
test_attack_dic["pod."] = 2 
test_attack_dic["portsweep."] = 1 
test_attack_dic["processtable."] = 2 
test_attack_dic["ps."] = 3 
test_attack_dic["rootkit."] = 3 
test_attack_dic["saint."] = 1 
test_attack_dic["satan."] = 1 
test_attack_dic["sendmail."] = 4 
test_attack_dic["smurf."] = 2 
test_attack_dic["snmpgetattack."] = 4 
test_attack_dic["snmpguess."] = 4 
test_attack_dic["sqlattack."] = 3 
test_attack_dic["teardrop."] = 2 
test_attack_dic["udpstorm."] = 2 
test_attack_dic["warezmaster."] = 4 
test_attack_dic["worm."] = 4 
test_attack_dic["xlock."] = 4 
test_attack_dic["xsnoop."] = 4 
test_attack_dic["xterm."] = 3
test_attack_dic['normal.'] = 0
attack_dic_num = {}
attack_dic_num[0] = ['normal']
attack_dic_num[2] = attack_dic['dos']
attack_dic_num[4] = attack_dic['r2l']
attack_dic_num[3] = attack_dic['u2r']
attack_dic_num[1] = attack_dic['probe']

In [5]:
#anomaly category
category_label_binary = []
map(lambda label: map(lambda cate: category_label_binary.append(cate) if label in attack_dic_num[cate] else None,attack_dic_num.keys()),training['label'])
category_label_binary = np.array(category_label_binary)
category_label_binary[np.where((category_label_binary != 0))] = 1

print 'done'
test_category_label_binary = []
map(lambda label: test_category_label_binary.append(test_attack_dic[label]),testing['label'])
test_category_label_binary = np.array(test_category_label_binary)
test_category_label_binary[np.where((test_category_label_binary != 0))] = 1

done


In [43]:
#attack category
category_label = []
map(lambda label: map(lambda cate: category_label.append(cate) if label in attack_dic_num[cate] else None,attack_dic_num.keys()),training['label'])
category_label = np.array(category_label)
#category_label[np.where((category_label != 0))] = 1

print 'done'
test_category_label = []
map(lambda label: test_category_label.append(test_attack_dic[label]),testing['label'])
test_category_label = np.array(test_category_label)
#test_category_label[np.where((test_category_label != 0))] = 1

done


In [46]:
np.unique(category_label)

array([0, 1, 2, 3, 4])

In [146]:
15000*0.05

750.0

In [120]:
#anomaly index
training_anomaly_index_array= []
for j in range(3,20):
    training_index = []
    for name in ['dos', 'u2r', 'r2l', 'probe']:
        now_attack_type = attack_dic[name]
        now_attack_type_dic = {'normal':0}
        for i in range(len(now_attack_type)):
            now_attack_type_dic[now_attack_type[i]] = i+1
    
        for now_name in now_attack_type:
            #print len(np.where(training['label'] == now_name)[0].tolist())
            attack_len = len(np.where(training['label'] == now_name)[0].tolist())
            if attack_len > (1000*j):  
                #print attack_len     
                #training_index = training_index + np.where(training['label'] == now_name)[0][:(j*1000)].tolist()
                training_index = training_index + np.where(training['label'] == now_name)[0][np.random.randint(0,attack_len, (j*1000))].tolist()
            else:
                training_index = training_index + np.where(training['label'] == now_name)[0].tolist()
            

    training_index = training_index + np.where(category_label == 0)[0][:int(0.05*len(training_index))].tolist()    
    training_anomaly_index_array.append(training_index)



In [ ]:
#attack index
training_attack_index_array = []
for i in range(3,15):
    training_index = []
    for name in ['dos', 'u2r', 'r2l', 'probe']:   
        now_attack_type = attack_dic[name]
        now_attack_type_dic = {'normal':0}
        for j in range(len(now_attack_type)):
            now_attack_type_dic[now_attack_type[j]] = j+1
    
        for now_name in now_attack_type:
            attack_len = len(np.where(training['label'] == now_name)[0].tolist())
            #print attack_len, attack_len > (i*10000)
            if attack_len > (i*1000):  
        
                training_index = training_index + np.where(training['label'] == now_name)[0][np.random.randint(0,attack_len, i*1000)].tolist()
            else:
                training_index = training_index + np.where(training['label'] == now_name)[0].tolist()
           
    training_attack_index_array.append(training_index)

testing_index = np.where(test_category_label != 0)[0].tolist() 
training_key_index = np.where(category_label != 0)[0].tolist()
#training_index = training_index + np.where(category_label == 0)[0][:len(training_index)*1].tolist()
#testing_index = np.where(test_category_label == 1)[0].tolist() + np.where(test_category_label == 2)[0][:10000].tolist() \
#                + np.where(test_category_label == 3)[0].tolist() + np.where(test_category_label == 4)[0][:5000].tolist()

# Combine the string columns

In [ ]:
#anomaly key
str_combination_anomaly = []
map(lambda x: str_combination_anomaly.append((x[0]+'_'+x[1]+'_'+x[2])) if (x[0]+'_'+x[1]+'_'+x[2]) not in str_combination_anomaly else None,training[string_label_columns].values)

In [ ]:
#attack type key
str_combination_attack = []
map(lambda x: str_combination_attack.append((x[0]+'_'+x[1]+'_'+x[2])) if (x[0]+'_'+x[1]+'_'+x[2]) not in str_combination_attack else None,training.iloc[training_key_index][string_label_columns].values)

In [ ]:
#anomaly key sort
pro_ser_flag_count_anomaly = {}
for x in str_combination_anomaly:
    pro_ser_flag_count_anomaly[x] = 0
for line in training[string_label_columns].values:
    key = line[0]+'_'+line[1]+'_'+line[2]
    pro_ser_flag_count_anomaly[key] = pro_ser_flag_count_anomaly[key] + 1

In [ ]:
#attack key sort
pro_ser_flag_count_attack = {}
for x in str_combination_attack:
    pro_ser_flag_count_attack[x] = 0
for line in training.iloc[training_key_index][string_label_columns].values:
    key = line[0]+'_'+line[1]+'_'+line[2]
    pro_ser_flag_count_attack[key] = pro_ser_flag_count_attack[key] + 1

In [ ]:
#anomlay top key 
sorted_dic_anomaly = sorted(pro_ser_flag_count_anomaly.iteritems(), key=lambda (k,v): (v,k),reverse=True)
new_string_columns_anomaly = []
new_string_columns_dic_anomaly = {}
loc_value = 0
for string in sorted_dic_anomaly:
    if (string[1] > 1) & (len(new_string_columns_anomaly) < 100):
        new_string_columns_anomaly.append(string[0])
        new_string_columns_dic_anomaly[string[0]] = loc_value
        loc_value = loc_value+1
    else:
        new_string_columns_anomaly.append('others')
        new_string_columns_dic_anomaly['others'] = loc_value
        break
inv_new_string_columns_dic_anomaly = {v: k for k, v in new_string_columns_dic_anomaly.items()}

In [ ]:
#attack top key 
sorted_dic_attack = sorted(pro_ser_flag_count_attack.iteritems(), key=lambda (k,v): (v,k),reverse=True)
new_string_columns_attack = []
new_string_columns_dic_attack = {}
loc_value = 0
for string in sorted_dic_attack:
    if (string[1] > 1) & (len(new_string_columns_attack) < 100):
        new_string_columns_attack.append(string[0])
        new_string_columns_dic_attack[string[0]] = loc_value
        loc_value = loc_value+1
    else:
        new_string_columns_attack.append('others')
        new_string_columns_dic_attack['others'] = loc_value
        break
inv_new_string_columns_dic_attack = {v: k for k, v in new_string_columns_dic_attack.items()}

# Creating columns

In [ ]:
#anomaly column
training_anomaly_copy = training.copy()
testing_anomaly_copy = testing.copy()
for column_name in new_string_columns_anomaly:
    training_anomaly_copy[column_name] = np.zeros(len(training))
    testing_anomaly_copy[column_name] = np.zeros(len(testing))
string_vector_anomaly = np.zeros([len(training_anomaly_copy),len(new_string_columns_anomaly)])
testing_string_vector_anomlay = np.zeros([len(testing_anomaly_copy),len(new_string_columns_anomaly)])
string_vector_loc_anomaly = []
testing_string_vector_loc_anomaly = []
map(lambda x: string_vector_loc_anomaly.append(new_string_columns_dic_anomaly[(x[0]+'_'+x[1]+'_'+x[2])]) if (x[0]+'_'+x[1]+'_'+x[2]) in new_string_columns_anomaly \
    else string_vector_loc_anomaly.append(new_string_columns_dic_anomaly['others']) ,training_anomaly_copy[string_label_columns].values)
map(lambda x: testing_string_vector_loc_anomaly.append(new_string_columns_dic_anomaly[(x[0]+'_'+x[1]+'_'+x[2])]) if (x[0]+'_'+x[1]+'_'+x[2]) in new_string_columns_anomaly \
    else testing_string_vector_loc_anomaly.append(new_string_columns_dic_anomaly['others']) ,testing_anomaly_copy[string_label_columns].values)

In [ ]:
#attack column
training_attack_copy = training.copy()
testing_attack_copy = testing.copy()
for column_name in new_string_columns_attack:
    training_attack_copy[column_name] = np.zeros(len(training))
    testing_attack_copy[column_name] = np.zeros(len(testing))
string_vector_attack = np.zeros([len(training_attack_copy),len(new_string_columns_attack)])
testing_string_vector_attack = np.zeros([len(testing_attack_copy),len(new_string_columns_attack)])
string_vector_loc_attack = []
testing_string_vector_loc_attack = []
map(lambda x: string_vector_loc_attack.append(new_string_columns_dic_attack[(x[0]+'_'+x[1]+'_'+x[2])]) if (x[0]+'_'+x[1]+'_'+x[2]) in new_string_columns_attack \
    else string_vector_loc_attack.append(new_string_columns_dic_attack['others']) ,training_attack_copy[string_label_columns].values)
map(lambda x: testing_string_vector_loc_attack.append(new_string_columns_dic_attack[(x[0]+'_'+x[1]+'_'+x[2])]) if (x[0]+'_'+x[1]+'_'+x[2]) in new_string_columns_dic_attack \
    else testing_string_vector_loc_attack.append(new_string_columns_dic_attack['others']) ,testing_attack_copy[string_label_columns].values)

In [ ]:
def replace_value(training_normal_numpy, key, loc):
    training_normal_numpy[loc][key+38] = 1
   

In [ ]:
#anomaly delete
for i in range(len(string_label_columns)):   
    del(training_anomaly_copy[string_label_columns[i]])
    del(testing_anomaly_copy[string_label_columns[i]])
del training_anomaly_copy['label']
del testing_anomaly_copy['label']


In [ ]:
#attack delete
for i in range(len(string_label_columns)):
    del(training_attack_copy[string_label_columns[i]])
    del(testing_attack_copy[string_label_columns[i]])
del training_attack_copy['label']
del testing_attack_copy['label']

In [ ]:
#anomaly replace
training_numpy_anomaly = training_anomaly_copy.values
testing_numpy_anomaly = testing_anomaly_copy.values
map(lambda i: replace_value(training_numpy_anomaly, string_vector_loc_anomaly[i], i),range(len(training_anomaly_copy)))
map(lambda i: replace_value(testing_numpy_anomaly, testing_string_vector_loc_anomaly[i],i),range(len(testing_anomaly_copy)))


In [ ]:
#attack replace
training_numpy_attack = training_attack_copy.values
testing_numpy_attack = testing_attack_copy.values
map(lambda i: replace_value(training_numpy_attack, string_vector_loc_attack[i], i),range(len(training_attack_copy)))
map(lambda i: replace_value(testing_numpy_attack, testing_string_vector_loc_attack[i], i),range(len(testing_attack_copy)))

In [ ]:
def pca_transform(training_numpy, testing_numpy, index, pca_dim, training_copy):
    
    min_max = preprocessing.MinMaxScaler()
    min_max.fit(training_numpy)
    x_scaled = min_max.fit_transform(training_numpy[index])
    testting_x_scaled = min_max.transform(testing_numpy)
    training_norm = pd.DataFrame(x_scaled, columns = training_copy.columns)
    testing_norm = pd.DataFrame(testting_x_scaled, columns = training_copy.columns)
    
    pca_model = PCA(n_components=pca_dim)
    training_pca_value = pca_model.fit_transform(training_norm)
    testing_pca_value = pca_model.transform(testing_norm)
    return (training_pca_value, testing_pca_value)

In [141]:
#anomaly training 
n_estimators = 10
start = time.time()
svc = SVC(C=pow(10,4))
anomoly_model = OneVsOneClassifier(BaggingClassifier(svc, n_estimators=n_estimators), n_jobs=-1)
(training_pca_value_anomaly, testing_pca_value_anomaly) = pca_transform(training_numpy_anomaly, testing_numpy_anomaly, training_anomaly_index_array[0],25, training_anomaly_copy)
anomoly_model.fit(training_pca_value_anomaly, category_label_binary[training_anomaly_index_array[0]])
anomaly_result = anomoly_model.predict(testing_pca_value_anomaly)
anomaly_index = np.where(anomaly_result == 1)[0].tolist()

/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


In [130]:
#attack training 
n_estimators = 10
start = time.time()
svc = SVC(C=pow(10,4))
attack_model = OneVsRestClassifier(BaggingClassifier(svc, n_estimators=n_estimators), n_jobs=-1)
(training_pca_value_attack, testing_pca_value_attack) = pca_transform(training_numpy_attack, testing_numpy_attack, training_attack_index_array[8],5, training_attack_copy)
attack_model.fit(training_pca_value_attack, category_label[training_attack_index_array[8]])
anomaly_result[anomaly_index] = attack_model.predict(testing_pca_value_attack[anomaly_index])

/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/usr/local

In [131]:
sum(anomaly_result == test_category_label) / float(len(anomaly_result))

0.93239215635841033

In [132]:
confu(anomaly_result, test_category_label)

array([[ 46015,      5,    284,      7,     83],
       [   636,   3893,    825,    149,    105],
       [  1130,    167, 228256,      5,   3932],
       [   656,      8,    106,     17,    249],
       [ 12156,     93,    382,     50,  11820]])

In [142]:
confu(anomaly_result, test_category_label_binary)

array([[ 46015,    379],
       [ 14578, 250057]])

In [139]:
confu(attack_model.predict(testing_pca_value_attack[testing_index]), test_category_label[testing_index])

array([[  3893,    871,    151,    115],
       [   169, 228410,      5,   3943],
       [     8,    106,     18,    261],
       [    96,    466,     54,  11870]])

In [138]:
len(attack_model.predict(testing_pca_value_attack)), len(test_category_label[testing_index])

(311029, 250436)

In [140]:
attack_model.score(testing_pca_value_attack[testing_index],test_category_label[testing_index])

0.97506348927470488

In [143]:
sum(anomaly_result == test_category_label_binary) / float(len(anomaly_result))

0.95191123657279542

In [ ]:
1